## Simple CSV Generation with Generation Counts

This section creates CSV files similar to the langfuse_export section 3, but adds a column for the number of generation attempts used for each trace.

In [1]:
import traceback
import pandas as pd
import json
import os
from datetime import datetime
from dateutil import parser

# Setup paths - same as langfuse_export
parent_dir = os.path.dirname(os.getcwd())
raw_export_dir = os.path.join(parent_dir, "raw_export")
processed_data_dir = os.path.join(parent_dir, "processed_data")


# Get session id list from data directory
session_id_list = []

for root, dirs, files in os.walk(raw_export_dir):
    for file in files:
        file_path = os.path.join(root, file)
        if "trimmed_" in file_path:
            session_id = file_path.split('trimmed_')[1].rstrip('.json')
            session_id_list.append(session_id)

print(f"Processing sessions: {session_id_list}")
print(f"Looking for raw files in: {raw_export_dir}")
print(f"Will save CSV files to: {processed_data_dir}")


def json_to_csv_weighted(session_id):
    """
    Convert JSON trace data to CSV format with aggregated metrics.
    Upgraded version that includes generation_count column.

    Args:
        session_id (str): Identifier for the session to process
    """

    def extract_observation_details(observations, trace_id):
        """Extract and aggregate metrics from observations"""
        metrics = {
            "status": None,
            "latency": 0,
            "total_tokens": 0,
            "prompt_tokens": 0,
            "completion_tokens": 0,
            "total_cost": 0,
            "input_cost": 0,
            "output_cost": 0,
            "parameters": set(),
            "generation_count": 0,  # New field for generation count
        }

        # Count generations and process GENERATION observations
        for obs in (o for o in observations if o["type"] == "GENERATION"):
            metrics["generation_count"] += 1
            metrics["total_tokens"] += obs["totalTokens"]
            metrics["prompt_tokens"] += obs["promptTokens"]
            metrics["completion_tokens"] += obs["completionTokens"]
            metrics["latency"] += obs["latency"]
            for key, value in obs["modelParameters"].items():
                metrics["parameters"].add(key + ":" + value)

            # Add costs if present
            for cost_type in ["Total", "Input", "Output"]:
                key = f"calculated{cost_type}Cost"
                metric_key = cost_type.lower() + "_cost"
                if obs.get(key) is not None:
                    metrics[metric_key] += obs[key]
                    
        if len(metrics["parameters"]) == 0:
            metrics["parameters"] = "N/A"
            
        # Process SPAN observations for status
        status_indicators = [
            obs["name"]
            for obs in observations
            if obs["type"] == "SPAN" and "start_" not in obs["name"]
        ]

        #  if later than 2025-05-19, use status_signal_from_output
        if datetime.now() > datetime(2025, 5, 19):
            pass
        else:
            # Determine status
            success_signals = sum("end_" in name for name in status_indicators)
            failure_signals = sum(
                "failure_signal" in name for name in status_indicators
            )

            if success_signals + failure_signals > 1:
                raise ValueError(
                    f"Multiple status indicators found in trace {trace_id}"
                )

            metrics["status"] = (
                "success"
                if success_signals
                else "failure" if failure_signals else "unknown"
            )

        metrics["prompt_cost"] = metrics.pop("input_cost")
        metrics["completion_cost"] = metrics.pop("output_cost")
        metrics["latency"] = round(metrics["latency"] / 1000, 2)
        return metrics

    def cal_time(trace):
        time_diff = datetime.fromisoformat(
            trace["updatedAt"].replace("Z", "+00:00")
        ) - datetime.fromisoformat(trace["createdAt"].replace("Z", "+00:00"))
        seconds_diff = time_diff.total_seconds()
        return seconds_diff

    try:
        if session_id.startswith("da0a"):
            session_id = "phi4_" + session_id
        simple_session_id = session_id.rsplit("_", 2)[0]

        # Load JSON data
        if "tpsg" in session_id:
            session_id_ = session_id.replace("tpsg", "tpusg")
        else:
            session_id_ = session_id
        trimmed_path = os.path.join(raw_export_dir, f"trimmed_{session_id_}.json")
        print(
            f"Processing session {session_id}, simple id {simple_session_id}. Look for {trimmed_path}"
        )
        with open(trimmed_path, "r") as file:
            traces = json.load(file)["data"]

        # Process traces
        rows = [
            {
                "num_run": trace["metadata"]["num_run"],
                "name": trace["name"],
                "trace_id": trace["id"],
                "batch_id": trace["session_id"],
                # "latency": cal_time(trace),
                # "latency": round(trace["latency"], 2),
                **extract_observation_details(
                    trace["observations"],
                    trace["id"],
                ),
                "status": (
                    "failure"
                    if trace["output"]["status"].lower() == "failed"
                    else "success"
                ),
                "tags": trace["tags"],
                "timestamp": int(parser.isoparse(trace["timestamp"]).timestamp()),
            }
            for trace in traces
        ]
        
        # Create and save DataFrame
        df = pd.DataFrame(rows).sort_values("num_run")

        output_dir = os.path.join(processed_data_dir, f"{simple_session_id}")
        if not os.path.exists(output_dir):
            os.makedirs(output_dir, exist_ok=True)

        output_path = os.path.join(output_dir, f"clean_{session_id_}.csv")

        print(output_path)
        df.to_csv(output_path, index=False)
        print(f"Successfully saved CSV to: {output_path}")

    except FileNotFoundError as e:
        print(
            f"FileNotFoundError: For session {session_id} not found. Looked for {trimmed_path}\nError info: \n{e}\n\nTraceback: {traceback.format_exc()}"
        )
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in input file for session {session_id}")
    except Exception as e:
        print(f"Error processing session {session_id}: {str(e)}")



# Example usage
for session_id in session_id_list:
    json_to_csv_weighted(session_id)

Processing sessions: ['qwen2.5-coder:32b_17ec_psg_batch', 'qwen2.5-coder:32b_17ec_tpusg_batch']
Looking for raw files in: /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.03/raw_export
Will save CSV files to: /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.03/processed_data
Processing session qwen2.5-coder:32b_17ec_psg_batch, simple id qwen2.5-coder:32b_17ec. Look for /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.03/raw_export/trimmed_qwen2.5-coder:32b_17ec_psg_batch.json
/Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.03/processed_data/qwen2.5-coder:32b_17ec/clean_qwen2.5-coder:32b_17ec_psg_batch.csv
Successfully saved CSV to: /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.03/processed_data/qwen2.5-coder:32b_17ec/clean_qwen2.5-coder:32b_17ec_psg_batch.csv
Processing session qwen2.5-coder:32b_17ec_tpusg_batch, simple id qwen2.5-coder:32b_17ec. Look for /Us

## Weighted Success Rate Analysis

Now let's analyze the data to compare traditional success rates vs generation-weighted success rates between PSG and TPUSG approaches.

In [2]:
# Load and analyze the generated CSV files
import glob
import numpy as np
import matplotlib.pyplot as plt


STAGE_ORDER = [
    "Data Processor",
    "Model Converter",
    "Sketch Generator",
    "PySketch Generator",
    "TPUSketch Generator",
]
STAGE_ABBR = {
    "Data Processor": "DP",
    "Model Converter": "MC",
    "Sketch Generator": "SG",
    "PySketch Generator": "PSG",
    "TPUSketch Generator": "TPUSG",
}
# Dfined for plottings
TASK_COLORS = {
    "Model Converter": "blue",
    "Data Processor": "green",
    "Sketch Generator": "purple",
    "PySketch Generator": "red",
    "TPUSketch Generator": "orange",
}

processor_names = {
    "dp": {
        "full_name": "Data Processor",
        "short_name": "DP",
        "processor_type": "data_processor",
    },
    "mc": {
        "full_name": "Model Converter",
        "short_name": "MC",
        "processor_type": "model_converter",
    },
    "sg": {
        "full_name": "Sketch Generator",
        "short_name": "SG",
        "processor_type": "sketch_generator",
    },
    "psg": {
        "full_name": "PySketch Generator",
        "short_name": "PSG",
        "processor_type": "py_sketch_generator",
    },
    "tpusg": {
        "full_name": "TPUSketch Generator",
        "short_name": "TPUSG",
        "processor_type": "tpu_sketch_generator",
    },
}

# Find all generated CSV files
csv_files = glob.glob(os.path.join(processed_data_dir, "**/clean_*.csv"), recursive=True)
print(f"Found {len(csv_files)} CSV files:")
for file in csv_files:
    print(f"  {file}")

# Load and combine data
all_data = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    # Extract session info from filename
    filename = os.path.basename(csv_file)
    session_id = filename.replace("weighted_", "").replace(".csv", "")
    df['session_id'] = session_id
    
    # Extract processor type
    if 'psg' in session_id.lower():
        df['processor_type'] = 'PSG'
    elif 'tpusg' in session_id.lower():
        df['processor_type'] = 'TPUSG'
    else:
        df['processor_type'] = 'Other'
    
    all_data.append(df)

if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"\nCombined dataset: {len(combined_df)} total records")
    print(f"Columns: {list(combined_df.columns)}")
    print(f"Processor types: {combined_df['processor_type'].value_counts().to_dict()}")
else:
    print("No data found. Please run the CSV generation first.")
    combined_df = pd.DataFrame()

Found 2 CSV files:
  /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.03/processed_data/qwen2.5-coder:32b_17ec/clean_qwen2.5-coder:32b_17ec_tpusg_batch.csv
  /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.03/processed_data/qwen2.5-coder:32b_17ec/clean_qwen2.5-coder:32b_17ec_psg_batch.csv

Combined dataset: 60 total records
Columns: ['num_run', 'name', 'trace_id', 'batch_id', 'status', 'latency', 'total_tokens', 'prompt_tokens', 'completion_tokens', 'total_cost', 'parameters', 'generation_count', 'prompt_cost', 'completion_cost', 'tags', 'timestamp', 'session_id', 'processor_type']
Processor types: {'TPUSG': 30, 'PSG': 30}


In [3]:
if not combined_df.empty:
    print("=== Detailed Success Rate Analysis ===")
    
    # Analysis by processor type - dynamically detect available processors
    analysis_results = {}
    
    # Get all unique processor types from the data
    available_processors = combined_df['processor_type'].unique()
    print(f"Available processors: {list(available_processors)}")
    
    for processor in available_processors:
        processor_data = combined_df[combined_df['processor_type'] == processor]
        if processor_data.empty:
            continue
            
        success_data = processor_data[processor_data['status'] == 'success']
        
        # Basic metrics
        total_runs = len(processor_data)
        success_count = len(success_data)
        traditional_success_rate = success_count / total_runs if total_runs > 0 else 0
        
        # Generation analysis for successful runs
        if not success_data.empty:
            avg_generations = success_data['generation_count'].mean()
            std_generations = success_data['generation_count'].std()
            min_generations = success_data['generation_count'].min()
            max_generations = success_data['generation_count'].max()
            
            # Generation distribution
            gen_distribution = success_data['generation_count'].value_counts().sort_index()
            
            # Weighted success rates
            # Method 1: Efficiency-weighted (1/generation_count)
            efficiency_scores = 1 / success_data['generation_count']
            efficiency_weighted_rate = efficiency_scores.sum() / total_runs
            
            # Method 2: Exponential penalty
            exponential_scores = np.exp(-0.5 * (success_data['generation_count'] - 1))
            exponential_weighted_rate = exponential_scores.sum() / total_runs
            
            # Method 3: Linear penalty
            linear_scores = 1 - 0.2 * (success_data['generation_count'] - 1)  # 20% penalty per extra generation
            linear_scores = np.maximum(linear_scores, 0.1)  # Minimum score of 0.1
            linear_weighted_rate = linear_scores.sum() / total_runs
            
            # Method 4: Robust/Rank-based (step/bucket approach)
            def robust_score(gen_count):
                if gen_count <= 2:  # First 2 attempts = full credit
                    return 1.0
                elif gen_count <= 4:  # 3-4 attempts = partial credit
                    return 0.6
                elif gen_count <= 6:  # 5-6 attempts = lower credit
                    return 0.3
                else:  # 7+ attempts = minimal credit
                    return 0.1
            
            robust_scores = success_data['generation_count'].apply(robust_score)
            robust_weighted_rate = robust_scores.sum() / total_runs
            
        else:
            avg_generations = std_generations = min_generations = max_generations = 0
            gen_distribution = pd.Series()
            efficiency_weighted_rate = exponential_weighted_rate = linear_weighted_rate = robust_weighted_rate = 0
        
        # Store results
        analysis_results[processor] = {
            'total_runs': total_runs,
            'success_count': success_count,
            'traditional_success_rate': traditional_success_rate,
            'avg_generations': avg_generations,
            'std_generations': std_generations,
            'min_generations': min_generations,
            'max_generations': max_generations,
            'generation_distribution': gen_distribution,
            'efficiency_weighted_rate': efficiency_weighted_rate,
            'exponential_weighted_rate': exponential_weighted_rate,
            'linear_weighted_rate': linear_weighted_rate,
            'robust_weighted_rate': robust_weighted_rate
        }
        
        # Print detailed results
        print(f"\n--- {processor} Analysis ---")
        print(f"Total runs: {total_runs}")
        print(f"Successful runs: {success_count}")
        print(f"Traditional success rate: {traditional_success_rate:.3f} ({traditional_success_rate*100:.1f}%)")
        
        if not success_data.empty:
            print(f"\nGeneration Analysis (successful runs only):")
            print(f"  Average generations: {avg_generations:.2f} ± {std_generations:.2f}")
            print(f"  Generation range: {min_generations} - {max_generations}")
            print(f"  Generation distribution:")
            for gen_count, freq in gen_distribution.items():
                percentage = freq / len(success_data) * 100
                print(f"    {gen_count} generation(s): {freq} runs ({percentage:.1f}%)")
            
            print(f"\nWeighted Success Rates:")
            print(f"  Efficiency-weighted: {efficiency_weighted_rate:.3f} ({efficiency_weighted_rate*100:.1f}%)")
            print(f"  Exponential-weighted: {exponential_weighted_rate:.3f} ({exponential_weighted_rate*100:.1f}%)")
            print(f"  Linear-weighted: {linear_weighted_rate:.3f} ({linear_weighted_rate*100:.1f}%)")
            print(f"  Robust-weighted: {robust_weighted_rate:.3f} ({robust_weighted_rate*100:.1f}%)")
        else:
            print("No successful runs to analyze.")
    
    # Comparative analysis - now handles any number of processors
    processor_list = list(analysis_results.keys())
    num_processors = len(processor_list)
    
    if num_processors >= 2:
        print(f"\n=== Comparative Analysis ({num_processors} processors) ===")
        
        # Show pairwise comparisons for all pairs of processors
        for i in range(num_processors):
            for j in range(i + 1, num_processors):
                proc1, proc2 = processor_list[i], processor_list[j]
                data1, data2 = analysis_results[proc1], analysis_results[proc2]
                
                print(f"\n--- {proc1} vs {proc2} Comparison ---")
                print(f"Traditional Success Rate:")
                print(f"  {proc1}: {data1['traditional_success_rate']:.3f} ({data1['traditional_success_rate']*100:.1f}%)")
                print(f"  {proc2}: {data2['traditional_success_rate']:.3f} ({data2['traditional_success_rate']*100:.1f}%)")
                print(f"  Difference: {(data2['traditional_success_rate'] - data1['traditional_success_rate']):.3f} ({(data2['traditional_success_rate'] - data1['traditional_success_rate'])*100:+.1f}%)")
                
                if data1['success_count'] > 0 and data2['success_count'] > 0:
                    print(f"\nAverage Generations (successful runs):")
                    print(f"  {proc1}: {data1['avg_generations']:.2f}")
                    print(f"  {proc2}: {data2['avg_generations']:.2f}")
                    print(f"  Difference: {(data2['avg_generations'] - data1['avg_generations']):+.2f}")
                    
                    for method_name, method_key in [
                        ("Efficiency-Weighted", "efficiency_weighted_rate"),
                        ("Exponential-Weighted", "exponential_weighted_rate"),
                        ("Linear-Weighted", "linear_weighted_rate"),
                        ("Robust-Weighted", "robust_weighted_rate")
                    ]:
                        print(f"\n{method_name} Success Rate:")
                        print(f"  {proc1}: {data1[method_key]:.3f} ({data1[method_key]*100:.1f}%)")
                        print(f"  {proc2}: {data2[method_key]:.3f} ({data2[method_key]*100:.1f}%)")
                        print(f"  Difference: {(data2[method_key] - data1[method_key]):.3f} ({(data2[method_key] - data1[method_key])*100:+.1f}%)")
                
                # Efficiency insight
                if data1['success_count'] > 0 and data2['success_count'] > 0:
                    eff1 = data1['efficiency_weighted_rate'] / data1['traditional_success_rate'] if data1['traditional_success_rate'] > 0 else 0
                    eff2 = data2['efficiency_weighted_rate'] / data2['traditional_success_rate'] if data2['traditional_success_rate'] > 0 else 0
                    
                    print(f"\nEfficiency Index (weighted/traditional ratio):")
                    print(f"  {proc1}: {eff1:.3f}")
                    print(f"  {proc2}: {eff2:.3f}")
                    print(f"  Higher efficiency index means fewer generations needed per success")
                    
                    if data2['traditional_success_rate'] > data1['traditional_success_rate']:
                        if data2['efficiency_weighted_rate'] > data1['efficiency_weighted_rate']:
                            print(f"\n✓ {proc2} is both more successful AND more efficient")
                        else:
                            print(f"\n⚠ {proc2} is more successful but less efficient (needs more generation attempts)")
                    else:
                        print(f"\n✓ {proc1} shows better performance")
    elif num_processors == 1:
        processor = processor_list[0]
        print(f"\n=== Single Processor Analysis ({processor}) ===")
        print("Only one processor type found in the data. No comparison available.")
        data = analysis_results[processor]
        
        print(f"\nSummary for {processor}:")
        print(f"  Total runs: {data['total_runs']}")
        print(f"  Success rate: {data['traditional_success_rate']:.3f} ({data['traditional_success_rate']*100:.1f}%)")
        if data['success_count'] > 0:
            print(f"  Average generations: {data['avg_generations']:.2f}")
            print(f"  Efficiency-weighted rate: {data['efficiency_weighted_rate']:.3f} ({data['efficiency_weighted_rate']*100:.1f}%)")
    else:
        print("\nNo processors found in the data.")

else:
    print("No data available for analysis. Please run the CSV generation step first.")

=== Detailed Success Rate Analysis ===
Available processors: ['TPUSG', 'PSG']

--- TPUSG Analysis ---
Total runs: 30
Successful runs: 30
Traditional success rate: 1.000 (100.0%)

Generation Analysis (successful runs only):
  Average generations: 1.70 ± 1.02
  Generation range: 1 - 5
  Generation distribution:
    1 generation(s): 17 runs (56.7%)
    2 generation(s): 8 runs (26.7%)
    3 generation(s): 3 runs (10.0%)
    4 generation(s): 1 runs (3.3%)
    5 generation(s): 1 runs (3.3%)

Weighted Success Rates:
  Efficiency-weighted: 0.748 (74.8%)
  Exponential-weighted: 0.777 (77.7%)
  Linear-weighted: 0.860 (86.0%)
  Robust-weighted: 0.923 (92.3%)

--- PSG Analysis ---
Total runs: 30
Successful runs: 12
Traditional success rate: 0.400 (40.0%)

Generation Analysis (successful runs only):
  Average generations: 1.75 ± 1.06
  Generation range: 1 - 4
  Generation distribution:
    1 generation(s): 7 runs (58.3%)
    2 generation(s): 2 runs (16.7%)
    3 generation(s): 2 runs (16.7%)
    4 

Available keys in analysis_results:
  TPUSG: ['total_runs', 'success_count', 'traditional_success_rate', 'avg_generations', 'std_generations', 'min_generations', 'max_generations', 'generation_distribution', 'efficiency_weighted_rate', 'exponential_weighted_rate', 'linear_weighted_rate', 'robust_weighted_rate']
  PSG: ['total_runs', 'success_count', 'traditional_success_rate', 'avg_generations', 'std_generations', 'min_generations', 'max_generations', 'generation_distribution', 'efficiency_weighted_rate', 'exponential_weighted_rate', 'linear_weighted_rate', 'robust_weighted_rate']



RELATIVE PERFORMANCE ANALYSIS

Success Rate Comparison Table:
Method                  TPUSG      PSG
--------------------------------------
Traditional            100.0%    40.0%
Efficiency
Weighted     74.8%    29.7%
Exponential
Weighted    77.7%    30.6%
Linear
Weighted         86.0%    34.0%
Robust
Weighted         92.3%    36.0%

TPUSG vs PSG Performance Ratios:
Traditional         : TPUSG 100.0% vs PSG  40.0% (Ratio: 2.50, TPUSG advantage)
Efficiency
Weighted : TPUSG  74.8% vs PSG  29.7% (Ratio: 2.52, TPUSG advantage)
Exponential
Weighted: TPUSG  77.7% vs PSG  30.6% (Ratio: 2.54, TPUSG advantage)
Linear
Weighted     : TPUSG  86.0% vs PSG  34.0% (Ratio: 2.53, TPUSG advantage)
Robust
Weighted     : TPUSG  92.3% vs PSG  36.0% (Ratio: 2.56, TPUSG advantage)

Analysis complete. Generated weighted CSV files are saved in: /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.03/processed_data


In [4]:
# Create comprehensive visualization - dynamic for any number of processors

# Get processors and colors dynamically
processors = list(analysis_results.keys())
num_processors = len(processors)

if num_processors == 0:
    print("No processors to visualize.")
else:
    # Generate distinct colors for all processors
    import matplotlib.cm as cm
    if num_processors <= 10:
        # Use a predefined color palette for small numbers
        color_palette = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'plum', 
                       'lightsalmon', 'lightsteelblue', 'khaki', 'lightpink', 'lightgray']
        colors = color_palette[:num_processors]
    else:
        # Use colormap for larger numbers
        colormap = cm.tab20 if num_processors <= 20 else cm.hsv
        colors = [colormap(i / num_processors) for i in range(num_processors)]
    
    fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(20, 12))
    
    # First, let's check what data we have
    print("Available keys in analysis_results:")
    for key, value in analysis_results.items():
        print(f"  {key}: {list(value.keys())}")
    print()
    
    # 1. Basic Success Rate Comparison
    success_rates = [analysis_results[p]['traditional_success_rate'] * 100 for p in processors]
    bars = ax1.bar(processors, success_rates, color=colors, alpha=0.7)
    ax1.set_ylabel('Success Rate (%)')
    ax1.set_title('Traditional Success Rate Comparison')
    ax1.grid(True, alpha=0.3)
    if num_processors > 3:
        ax1.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, rate in zip(bars, success_rates):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{rate:.1f}%', ha='center', va='bottom')
    
    # 2. Total Runs Comparison
    total_runs = [analysis_results[p]['total_runs'] for p in processors]
    bars = ax2.bar(processors, total_runs, color=colors, alpha=0.7)
    ax2.set_ylabel('Total Runs')
    ax2.set_title('Total Runs Comparison')
    ax2.grid(True, alpha=0.3)
    if num_processors > 3:
        ax2.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, runs in zip(bars, total_runs):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f'{runs}', ha='center', va='bottom')
    
    # 3. Success Count Comparison
    success_counts = [analysis_results[p]['success_count'] for p in processors]
    bars = ax3.bar(processors, success_counts, color=colors, alpha=0.7)
    ax3.set_ylabel('Successful Runs')
    ax3.set_title('Successful Runs Comparison')
    ax3.grid(True, alpha=0.3)
    if num_processors > 3:
        ax3.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, count in zip(bars, success_counts):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, 
                f'{count}', ha='center', va='bottom')
    
    # 4. Average Generations Comparison (only for processors with successful runs)
    processors_with_success = [p for p in processors if analysis_results[p]['success_count'] > 0]
    if processors_with_success:
        avg_gens = [analysis_results[p]['avg_generations'] for p in processors_with_success]
        std_gens = [analysis_results[p]['std_generations'] for p in processors_with_success]
        success_colors = [colors[processors.index(p)] for p in processors_with_success]
        
        bars = ax4.bar(processors_with_success, avg_gens, yerr=std_gens, capsize=5, 
                      color=success_colors, alpha=0.7)
        ax4.set_ylabel('Average Generations')
        ax4.set_title('Average Generation Attempts for Successful Runs')
        ax4.grid(True, alpha=0.3)
        if len(processors_with_success) > 3:
            ax4.tick_params(axis='x', rotation=45)
        
        # Add value labels on bars
        for bar, avg in zip(bars, avg_gens):
            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
                    f'{avg:.2f}', ha='center', va='bottom')
    else:
        ax4.text(0.5, 0.5, 'No successful runs found', 
                ha='center', va='center', transform=ax4.transAxes)
        ax4.set_title('Average Generation Attempts for Successful Runs')
    
    # 5. Generation Distribution for the first processor with successful runs
    first_successful_processor = None
    for processor in processors:
        if (processor in analysis_results and 
            'generation_distribution' in analysis_results[processor] and
            analysis_results[processor]['success_count'] > 0):
            first_successful_processor = processor
            break
    
    if first_successful_processor:
        gen_dist = analysis_results[first_successful_processor]['generation_distribution']
        if not gen_dist.empty:
            gen_counts = list(gen_dist.index)
            gen_values = list(gen_dist.values)
            processor_color = colors[processors.index(first_successful_processor)]
            ax5.bar(gen_counts, gen_values, color=processor_color, alpha=0.7)
            ax5.set_xlabel('Generation Attempts')
            ax5.set_ylabel('Count')
            ax5.set_title(f'{first_successful_processor} Generation Attempt Distribution')
            ax5.grid(True, alpha=0.3)
        else:
            ax5.text(0.5, 0.5, f'No {first_successful_processor} generation data available', 
                    ha='center', va='center', transform=ax5.transAxes)
            ax5.set_title(f'{first_successful_processor} Generation Attempt Distribution')
    else:
        ax5.text(0.5, 0.5, 'No generation data available', 
                ha='center', va='center', transform=ax5.transAxes)
        ax5.set_title('Generation Attempt Distribution')
    
    # 6. Weighted Success Rate Comparison
    methods = ['Traditional', 'Efficiency\nWeighted', 'Exponential\nWeighted', 'Linear\nWeighted', 'Robust\nWeighted']
    
    # Prepare data for all processors
    x = np.arange(len(methods))
    width = 0.8 / num_processors  # Adjust bar width based on number of processors
    
    # Plot bars for each processor
    for i, processor in enumerate(processors):
        if processor in analysis_results:
            data = analysis_results[processor]
            rates = [
                data.get('traditional_success_rate', 0) * 100,
                data.get('efficiency_weighted_rate', 0) * 100,
                data.get('exponential_weighted_rate', 0) * 100,
                data.get('linear_weighted_rate', 0) * 100,
                data.get('robust_weighted_rate', 0) * 100
            ]
            
            # Calculate position for this processor's bars
            pos = x + (i - num_processors/2 + 0.5) * width
            
            bars = ax6.bar(pos, rates, width, label=processor, 
                          color=colors[i], alpha=0.7)
            
            # Add value labels on bars
            for bar, rate in zip(bars, rates):
                if rate > 0:  # Only show label if there's a value
                    ax6.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                            f'{rate:.1f}%', ha='center', va='bottom', fontsize=8)
    
    ax6.set_xlabel('Scoring Method')
    ax6.set_ylabel('Success Rate (%)')
    ax6.set_title('Success Rate Comparison Across Weighting Methods')
    ax6.set_xticks(x)
    ax6.set_xticklabels(methods, rotation=45, ha='right')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate and display relative performance for all processor pairs
    print("\n" + "="*60)
    print("RELATIVE PERFORMANCE ANALYSIS")
    print("="*60)
    
    # Define method_keys for use in all cases
    method_keys = ['traditional_success_rate', 'efficiency_weighted_rate', 
                  'exponential_weighted_rate', 'linear_weighted_rate', 'robust_weighted_rate']
    
    if num_processors >= 2:
        # Show performance comparison table
        print(f"\nSuccess Rate Comparison Table:")
        print(f"{'Method':<20} " + " ".join([f"{p:>8}" for p in processors]))
        print("-" * (20 + 9 * num_processors))
        
        for i, (method_name, method_key) in enumerate(zip(methods, method_keys)):
            rates = [analysis_results[p][method_key] * 100 for p in processors]
            print(f"{method_name:<20} " + " ".join([f"{rate:>7.1f}%" for rate in rates]))
        
        # Show pairwise ratios if exactly 2 processors
        if num_processors == 2:
            proc1, proc2 = processors[0], processors[1]
            print(f"\n{proc1} vs {proc2} Performance Ratios:")
            for i, method in enumerate(methods):
                rate1 = analysis_results[proc1][method_keys[i]] * 100
                rate2 = analysis_results[proc2][method_keys[i]] * 100
                
                if rate2 > 0:
                    ratio = rate1 / rate2
                    advantage = proc1 if ratio > 1 else proc2
                    print(f"{method:20}: {proc1} {rate1:5.1f}% vs {proc2} {rate2:5.1f}% "
                          f"(Ratio: {ratio:.2f}, {advantage} advantage)")
                else:
                    print(f"{method:20}: {proc1} {rate1:5.1f}% vs {proc2} {rate2:5.1f}% "
                          f"({proc2} has no successful runs)")
    
    elif num_processors == 1:
        processor = processors[0]
        data = analysis_results[processor]
        print(f"\nSingle Processor Summary ({processor}):")
        for i, method in enumerate(methods):
            rate = data[method_keys[i]] * 100
            print(f"{method:20}: {rate:5.1f}%")
    
    print(f"\nAnalysis complete. Generated weighted CSV files are saved in: {processed_data_dir}")